# Data Cleaning
In this project, I'll build a customer model that segments customers as an active or inactive customer (subscribed vs unsubscribed) based on of data from BoomBrush, a Dutch e-commerce site. The objective of this notebook is to clean the data of BoomBrush in order to analayse the data.

Why look at customer subscriptions? It's more cost effective to retain existing customers than to acquire new ones, which is why it's important to track customers at high risk of turnover (unsubscribers) and target them with retention strategies.

### Load the data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

from pandas.util import hash_pandas_object

In [2]:
# Read data
orders = pd.read_excel('../data/orders.xlsx')
customers = pd.read_excel('../data/subscriptions.xlsx')

In [3]:
orders.head(1)

,OrderId,CustomerId,Subscription,Refill,Status,PaidDate,PaidTime,WeekNumber,PaymentMethod,TBDUO,TBBLK,TBWHT,TBHBLK,TBHWHT,CCWHT,CCBLK,CodeUsed,Country,Shipping,Total
0,2253,772.0,N,Y,pending,05-07-2020,02:07:02,27.0,directdebit,0,0,0,1,0,0,0,NaN,Nederland,0.0,5.0


In [4]:
customers.head(1)

,FirstName,Email,UserId,SubscriptionId,Active,Interval,StartDate,BHBLK [Count total purchase],BHWHT,Price per interval,TBDUO[Non subscription purchase],TBBLK,TBWHT,TBHBLK,TBHWHT,CCWHT,CCBLK,EndDate,StopReason,StopReasonExtraMessage
0,NaN,NaN,1342,995,Y,3,01-07-2020,0,0,10.0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [5]:
stopreason = customers[customers["StopReason"].isna()==False]
stopreason.head(5)

,FirstName,Email,UserId,SubscriptionId,Active,Interval,StartDate,BHBLK [Count total purchase],BHWHT,Price per interval,TBDUO[Non subscription purchase],TBBLK,TBWHT,TBHBLK,TBHWHT,CCWHT,CCBLK,EndDate,StopReason,StopReasonExtraMessage
72,NaN,NaN,1212,923,N,3,10-05-2020,0,0,5.0,0,0,0,0,0,0,0,2020-05-21T23:00:00.000000Z,Ik vind het product niet prettig.,NaN
83,NaN,NaN,1199,912,N,1,09-05-2020,0,0,5.0,0,0,0,0,0,0,0,2020-05-12T23:00:00.000000Z,Ik heb nog borstelkopjes,NaN
232,NaN,NaN,1029,763,N,2,29-04-2020,0,0,5.0,0,0,0,0,0,0,0,2020-04-28T23:00:00.000000Z,Ik heb nog borstelkopjes,NaN
367,NaN,NaN,861,628,N,1,17-04-2020,0,0,5.0,0,0,0,0,0,0,0,2020-04-29T23:00:00.000000Z,Other,ik bestel ze zelf
400,NaN,NaN,821,595,N,1,15-04-2020,1,0,5.0,0,0,0,0,0,0,0,2020-05-17T23:00:00.000000Z,Ik heb nog borstelkopjes,NaN


In [6]:
print(orders.shape)
print(customers.shape)

(2023, 20)
(967, 20)


In [7]:
## Clean Customer Data

# Change UserId to string
customers["UserId"] = customers["UserId"].astype(str)

# Rename column names
customers = customers.rename(columns={"UserId":"CustomerId",
                                     "BHBLK [Count total purchase]":"BHBLK",
                                     "TBDUO[Non subscription purchase]":"TBDUO",
                                     "BHWHT ":"BHWHT"})

# Replacing Y/N for 0/1 - 1 meaning the customer is inactive/churned/dead
dictionary = {
    'Y': '0',
    'N': '1'
}
customers["Active"].replace(dictionary, inplace=True)

# Filter only Subscribers
customers = customers[customers["SubscriptionId"].isna()==False]
customers.shape

# Drop unnecessary columns - Customers
drop_cols = ["FirstName","Email","SubscriptionId",
            'BHBLK','BHWHT','TBDUO','TBBLK','TBWHT','TBHBLK','TBHWHT','CCWHT','CCBLK']
customers.drop(drop_cols, axis=1, inplace=True)

# Check data
customers.head(5)

,CustomerId,Active,Interval,StartDate,Price per interval,EndDate,StopReason,StopReasonExtraMessage
0,1342,0,3,01-07-2020,10.0,NaN,NaN,NaN
1,1341,0,3,01-07-2020,5.0,NaN,NaN,NaN
2,1340,0,3,01-07-2020,5.0,NaN,NaN,NaN
3,1332,0,3,28-06-2020,10.0,NaN,NaN,NaN
4,1334,0,3,28-06-2020,5.0,NaN,NaN,NaN


In [8]:
# Check for na or missing data
orders.isna().sum()

# Remove rows that do not contain customerID
drop_rows = orders.loc[orders["CustomerId"].isnull()].index
orders = orders.drop(drop_rows, axis=0)

In [9]:
## Clean Order Data

# Select only paid/shipped orders
orders = orders[(orders["Status"]=="shipped")|
               (orders["Status"]=="paid")]

# Select only orders > 10 (excluding bloggers etc)
orders = orders[(orders["Total"]>10)]

# Calculate gross revenue (excl. shipping cost)
orders["Revenue"] = orders["Total"]-orders["Shipping"]

# Change UserId to string
orders["OrderId"] = orders["OrderId"].astype(str)
orders["CustomerId"] = orders["CustomerId"].astype(int)
orders["CustomerId"] = orders["CustomerId"].astype(str)

# Replacing Y/N for 1/0
dictionary = {
    "Y": "1",
    "N": "0"}
orders["Subscription"].replace(dictionary, inplace=True)
orders["Refill"].replace(dictionary, inplace=True)

# Rename CodeUsed
orders["CodeUsed"] = np.where(orders["CodeUsed"].isnull, 0, 1)

# Calculate quanity of order
orders["Quantity"] = orders[['TBDUO','TBBLK','TBWHT','TBHBLK','TBHWHT','CCWHT','CCBLK']].sum(axis=1)

# Drop unnecessary columns - Orders
drop_cols = ["PaidTime","WeekNumber","Total","Shipping","Status"]
orders.drop(drop_cols, axis=1, inplace=True)

# Check data
orders.head(5)

,OrderId,CustomerId,Subscription,Refill,PaidDate,PaymentMethod,TBDUO,TBBLK,TBWHT,TBHBLK,TBHWHT,CCWHT,CCBLK,CodeUsed,Country,Revenue,Quantity
3,2250,848,0,0,02-07-2020,ideal,0,0,1,0,0,0,0,0,Nederland,69.0,1
4,2249,1342,0,0,01-07-2020,ideal,1,0,0,0,0,0,0,0,Nederland,118.0,1
6,2247,1341,0,0,01-07-2020,ideal,0,0,1,0,0,0,0,0,Nederland,59.0,1
7,2246,1340,0,0,01-07-2020,ideal,0,0,1,0,0,0,0,0,Nederland,59.0,1
8,2245,1339,0,0,01-07-2020,ideal,1,0,0,0,0,0,0,0,Nederland,138.0,1


In [10]:
# # Calculate quanity of black products bought
# orders["Black"] = orders[['TBBLK','TBHBLK','CCBLK']].sum(axis=1)
# orders["White"] = orders[['TBWHT','TBHWHT','CCWHT']].sum(axis=1)
# orders["Black & White"] = orders[['TBDUO']].sum(axis=1)

# # Drop unnecessary columns
# orders.drop(['TBDUO','TBBLK','TBWHT','TBHBLK','TBHWHT','CCWHT','CCBLK'], axis=1, inplace=True)

# orders.sort_values(by="Black",ascending=False)

# # Create variable for all black products
# orders.loc[(orders["TBBLK"]>=1)|
#            (orders["TBHBLK"]>=1)|
#            (orders["CCBLK"]>=1),"BLK"]=1

# # Create variable for all white products
# orders.loc[(orders["TBWHT"]>=1)|
#            (orders["TBHWHT"]>=1)|
#            (orders["CCWHT"]>=1),"WHT"]=1

# # Create variable for all black & white products
# orders.loc[((orders["BLK"]>=1)&
#            (orders["WHT"]>=1))|
#            (orders["TBDUO"]>=1),"Products"]= "Duo"

# # Change variable for all black & white products
# orders.loc[(orders["BLK"]>=1)&
#            (orders["WHT"].isna()),"Products"]="Black"

# orders.loc[(orders["BLK"].isna())&
#            (orders["WHT"]>=1),"Products"]="White"

# Create variable for all brushes
orders.loc[(orders["TBBLK"]>=1)|
           (orders["TBWHT"]>=1)|
           (orders["TBDUO"]>=1),"Products"]="Brush"

# Create variable for all accessoires
orders.loc[(orders["TBHWHT"]>=1)|
           (orders["TBHBLK"]>=1),"Products"]="Heads"

orders.loc[(orders["CCBLK"]>=1)|
           (orders["CCWHT"]>=1),"Products"]="Cables"


# Drop unnecessary columns
orders.drop(['TBDUO','TBBLK','TBWHT','TBHBLK','TBHWHT','CCWHT','CCBLK'], axis=1, inplace=True)

orders.sort_values(by="Quantity",ascending=False)

,OrderId,CustomerId,Subscription,Refill,PaidDate,PaymentMethod,CodeUsed,Country,Revenue,Quantity,Products
624,1562,957,0,0,02-06-2020,ideal,0,Nederland,474.0,12,Brush
1819,353,191,0,0,24-02-2020,ideal,0,Nederland,40.0,8,Heads
1927,242,116,1,0,12-12-2019,ideal,0,Nederland,126.4,6,Heads
616,1570,1310,0,0,06-06-2020,paypal,0,Nederland,99.0,5,Heads
993,1191,1052,0,0,29-04-2020,ideal,0,Nederland,89.0,5,Heads
...,...,...,...,...,...,...,...,...,...,...,...
1119,1064,976,1,0,24-04-2020,ideal,0,Nederland,118.0,1,Brush
1121,1062,966,1,0,23-04-2020,ideal,0,Nederland,59.0,1,Brush
1122,1061,965,1,0,22-04-2020,ideal,0,Nederland,59.0,1,Brush
1123,1060,964,0,0,22-04-2020,ideal,0,Nederland,118.0,1,Brush


In [11]:
orders.shape

(1153, 11)

In [12]:
customers.shape

(967, 8)

In [13]:
# We create a seperate dataset to work with the RFM framework later on.
from lifetimes.utils import calibration_and_holdout_data

rfm = calibration_and_holdout_data(orders, 'OrderId', 'PaidDate',
calibration_period_end='2019-10-21',
observation_period_end='2020-07-01' )
rfm.head()


,frequency_cal,recency_cal,T_cal,frequency_holdout,duration_holdout
OrderId,,,,,
151,0.0,0.0,0.0,0.0,254
181,0.0,0.0,283.0,0.0,254
182,0.0,0.0,283.0,0.0,254
183,0.0,0.0,283.0,0.0,254
184,0.0,0.0,283.0,0.0,254


### Merging and cleaning the dataset

There are two columns used to identify customers. `OrderId` is a order ID token that is generated for every order. If the same customer makes multiple orders, he has multiple customer_id identifiers. What we want to use for this analysis is `CustomerId`, which is unique to each purchaser and can be used to track their purchases over time. 

We group the data based on CustomerID, as shown in the Entity Relationship Diagram below:
![EDR](../edr.png)

In [14]:
# Create lambda to join products bought
combine_text = lambda x: ", ".join(x.unique())

# Group customer data based customer id
data = orders.groupby(["CustomerId"]).agg({"PaidDate":np.min,
                                           "Revenue":np.sum,
                                           "Subscription":"last",
                                           "Refill":np.max,
                                           "PaymentMethod":"last",
                                           "CodeUsed":np.max,
                                           "Country":"last",
                                           "Quantity":np.sum,
#                                            "Black":np.sum,
#                                            "White":np.sum,
#                                            "Black & White":np.sum,
                                           "Products":'last',
                                           "OrderId":'nunique'}).reset_index()
data = data.rename(columns={"PaidDate":"LastOrderDate",
                           "OrderId":"Transactions"})

In [15]:
# Grouping by CustomerId
data = pd.merge(data,customers,on="CustomerId")

In [16]:
# Change datatype
data["LastOrderDate"] = pd.to_datetime(data["LastOrderDate"], format="%d-%m-%Y")

# Calculate today's date
data["Today"] = "2020-07-02"
data["Today"] = pd.to_datetime(data["Today"])

# Calculate days since last order and today
data["DaysSinceLastOrder"] = data["Today"] - data["LastOrderDate"]
data["DaysSinceLastOrder"] = data["DaysSinceLastOrder"].astype(str)
data["DaysSinceLastOrder"] = data["DaysSinceLastOrder"].str.split("days").str[0]

# # Rename to recency
# data = data.rename(columns={"DaysSinceLastOrder":"Recency"})

In [17]:
# Change datatypes - StartDate
data["StartDate"] = pd.to_datetime(data["StartDate"], format="%d-%m-%Y")

# Change datatype - EndDate
data["EndDate"] = pd.to_datetime(data["EndDate"]).dt.tz_localize(None)
data["EndDate"] = pd.to_datetime(data["EndDate"], format="%d-%m-%Y")

# Resource: https://stackoverflow.com/questions/59744589/how-can-i-convert-the-string-2020-01-06t000000-000z-to-datetime-in-python

# When EndDate is empty the customer is still subscribed so we fill the date with today's date
data["EndDate"] = data["EndDate"].fillna(data["Today"])

# Calculate the lifetime of a customer in days
data["SubscriptionLifetime"] = data["EndDate"] - data["StartDate"]
data["SubscriptionLifetime"] = data["SubscriptionLifetime"].astype(str)
data["SubscriptionLifetime"] = data["SubscriptionLifetime"].str.split("days").str[0]
data["SubscriptionLifetime"] = data["SubscriptionLifetime"].astype(int)

# Change days to 1
data.loc[(data["SubscriptionLifetime"]<=0),"SubscriptionLifetime"]=1

# Calculate days between first order and today
data["DaysSinceFirstOrder"] = data["Today"] - data["StartDate"]
data["DaysSinceFirstOrder"] = data["DaysSinceFirstOrder"].astype(str)
data["DaysSinceFirstOrder"] = data["DaysSinceFirstOrder"].str.split("days").str[0]
data["DaysSinceFirstOrder"] = data["DaysSinceFirstOrder"].astype(int)


In [18]:
# Check date range
print("First recorded date:",data["StartDate"].min())
print("Last recorded date:",data["StartDate"].max())

First recorded date: 2019-10-21 00:00:00
Last recorded date: 2020-07-01 00:00:00


In [19]:
# Drop unnecessary columns - Data
drop_cols = ["LastOrderDate","StartDate","EndDate","Today"]
data.drop(drop_cols, axis=1, inplace=True)

In [20]:
# Hash CustomerId
data["ID"] = hash_pandas_object(data["CustomerId"])
data = data.drop(["CustomerId"],axis=1)
data.sort_values(by="Transactions",ascending=False)

,Revenue,Subscription,Refill,PaymentMethod,CodeUsed,Country,Quantity,Products,Transactions,Active,Interval,Price per interval,StopReason,StopReasonExtraMessage,DaysSinceLastOrder,SubscriptionLifetime,DaysSinceFirstOrder,ID
440,300.0,1,0,creditcard,0,Nederland,5,Brush,5,1,3,10.0,NaN,NaN,247,246,246,14714824792989382846
817,251.0,1,1,ideal,0,Nederland,6,Brush,4,0,2,10.0,NaN,NaN,0,77,77,12158826644895990813
439,364.0,1,1,ideal,0,Nederland,11,Brush,4,0,3,10.0,NaN,NaN,86,105,105,13883411180810797937
819,276.0,1,1,ideal,0,Nederland,7,Brush,4,0,3,10.0,NaN,NaN,1,77,77,6719897400227932307
100,216.0,1,1,paypal,0,Nederland,6,Brush,3,0,2,10.0,NaN,NaN,61,61,61,7055538257474694359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,69.0,1,0,ideal,0,Nederland,1,Brush,1,0,3,5.0,NaN,NaN,115,115,115,736561800732140135
336,69.0,1,0,creditcard,0,Nederland,1,Brush,1,0,3,5.0,NaN,NaN,115,115,115,2320120290218914254
337,69.0,1,0,ideal,0,Nederland,1,Brush,1,0,3,5.0,NaN,NaN,114,114,114,16164724063276218987
338,69.0,1,0,ideal,0,Nederland,1,Brush,1,1,3,5.0,NaN,NaN,114,114,114,10480480552032908954


In [21]:
list(data.columns)

['Revenue',
 'Subscription',
 'Refill',
 'PaymentMethod',
 'CodeUsed',
 'Country',
 'Quantity',
 'Products',
 'Transactions',
 'Active',
 'Interval',
 'Price per interval',
 'StopReason',
 'StopReasonExtraMessage',
 'DaysSinceLastOrder',
 'SubscriptionLifetime',
 'DaysSinceFirstOrder',
 'ID']

In [22]:
# Change order of the columns
data = data[['ID','Subscription','Refill','Interval','Price per interval',
             'PaymentMethod','CodeUsed','Country',
             'Quantity','Products','Transactions','Revenue',
#              'Black','White','Black & White',
             'DaysSinceLastOrder','SubscriptionLifetime',
             'StopReason','StopReasonExtraMessage',"Active"]]

In [23]:
data.sort_values(by="Refill",ascending=False)

,ID,Subscription,Refill,Interval,Price per interval,PaymentMethod,CodeUsed,Country,Quantity,Products,Transactions,Revenue,DaysSinceLastOrder,SubscriptionLifetime,StopReason,StopReasonExtraMessage,Active
439,13883411180810797937,1,1,3,10.0,ideal,0,Nederland,11,Brush,4,364.0,86,105,NaN,NaN,0
100,7055538257474694359,1,1,2,10.0,paypal,0,Nederland,6,Brush,3,216.0,61,61,NaN,NaN,0
504,9401012618591247566,1,1,3,5.0,ideal,0,Nederland,6,Heads,2,74.0,17,102,NaN,NaN,0
819,6719897400227932307,1,1,3,10.0,ideal,0,Nederland,7,Brush,4,276.0,1,77,NaN,NaN,0
286,2394225225664363277,1,1,3,5.0,ideal,0,Nederland,7,Brush,3,143.0,16,106,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,15444894199192121144,1,0,3,5.0,ideal,0,Nederland,1,Brush,1,69.0,121,121,NaN,NaN,0
321,4458339540397840812,1,0,3,5.0,ideal,0,Nederland,1,Brush,1,69.0,121,121,NaN,NaN,0
322,8251802578618047696,1,0,3,5.0,ideal,0,Nederland,3,Cables,1,79.0,121,121,NaN,NaN,0
323,14356298231128096804,0,0,3,5.0,ideal,0,Nederland,1,Brush,1,69.0,121,121,NaN,NaN,1


In [24]:
data.shape

(945, 17)

In [25]:
data.describe()

,ID,Interval,Price per interval,CodeUsed,Quantity,Transactions,Revenue,SubscriptionLifetime
count,9.450000e+02,945.000000,945.000000,945.0,945.000000,945.000000,945.000000,945.000000
mean,9.330877e+18,2.815873,5.862434,0.0,1.266667,1.097354,71.877725,93.554497
std,5.326182e+18,0.409063,1.903976,0.0,0.877448,0.372582,38.947608,46.711378
min,1.386568e+16,1.000000,0.000000,0.0,1.000000,1.000000,13.800000,1.000000
25%,4.879608e+18,3.000000,5.000000,0.0,1.000000,1.000000,49.000000,64.000000
50%,9.443953e+18,3.000000,5.000000,0.0,1.000000,1.000000,59.000000,83.000000
75%,1.396028e+19,3.000000,5.000000,0.0,1.000000,1.000000,69.000000,104.000000
max,1.842788e+19,3.000000,10.000000,0.0,13.000000,5.000000,523.000000,255.000000


In [26]:
# Now that we have a cleaner dataset ready for machine learning, let's save it as a new csv
data.to_csv("../data/data.csv",index=False)

rfm.to_csv("../data/rfm.csv")